Relation extraction (RE) is the process of identifying semantic relationships between entities in text.

This section provides a comprehensive discussion of relation extraction techniques, challenges, tools, and examples.

Each point is followed by potential demonstration ideas that use code to illustrate the concept.



### 6.1 Definition and Goals

- **Definition**: Relation extraction is the task of detecting and classifying semantic relationships between named entities within a given text. Relations may represent factual associations, such as "Barack Obama was born in Honolulu."
  - **Code Demonstration**: You could use a sample sentence and NLTK to split the sentence into entities and identify possible relations.


In [3]:
import nltk
from nltk import word_tokenize, pos_tag


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Define the sentence to analyze
# This sentence contains the name "Barack Obama" and the place "Honolulu"
sentence = "Barack Obama was born in Honolulu."

# Tokenize the sentence into individual words
# word_tokenize splits the sentence into tokens (words and punctuation)
tokens = word_tokenize(sentence)

# Perform Part-of-Speech (POS) tagging on the tokens
# pos_tag assigns a POS tag to each token, indicating its grammatical role
pos_tags = pos_tag(tokens)

# Print the tokens and their corresponding POS tags
print(pos_tags)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('Barack', 'NNP'), ('Obama', 'NNP'), ('was', 'VBD'), ('born', 'VBN'), ('in', 'IN'), ('Honolulu', 'NNP'), ('.', '.')]


- **Goal**: Extract structured relationships between entities (such as "Person-BornIn-Location") to enable effective querying of information from unstructured text.
  - **Code Demonstration**: A basic rule-based approach could be demonstrated to detect "Person-BornIn-Location" relationships.


In [4]:
import nltk
from nltk import word_tokenize, pos_tag

# Define a function to extract relationships from a given sentence
def extract_relation(sentence):
    # Check if the phrase "was born in" is present in the sentence
    if "was born in" in sentence:
        # Tokenize the sentence into individual words
        tokens = word_tokenize(sentence)

        # Perform Part-of-Speech (POS) tagging on the tokens
        # Filter for proper nouns (NNP), as these likely represent entities (e.g., people, locations)
        entities = [token for token, pos in pos_tag(tokens) if pos == 'NNP']

        # If there are at least two proper noun entities, return the relationship
        # The assumption is that the first entity is the person and the second is the location
        if len(entities) >= 2:
            return (entities[0], "born_in", entities[-1])

# Define a sample sentence
sentence = "Barack Obama was born in Honolulu."

# Extract the relationship from the sentence
relation = extract_relation(sentence)

# Print the extracted relationship
print(relation)


('Barack', 'born_in', 'Honolulu')


### 6.2 Types of Relations

- **Explicit Relations**: Relationships explicitly stated in the text (e.g., "CEO of Microsoft").
  - **Code Demonstration**: Extract explicit relations by matching specific patterns in a sentence.


In [5]:
import re

# Define the text to analyze
# The sentence contains the name of a person, their role, and the organization they work for
text = "Satya Nadella is the CEO of Microsoft."

# Define a regular expression pattern to extract the person, role, and organization
# The pattern uses named groups to capture specific entities
pattern = r'(?P<person>\w+ \w+) is the (?P<role>CEO) of (?P<org>\w+)'

# Search for the pattern in the text
match = re.search(pattern, text)

# If a match is found, print the named groups as a dictionary
if match:
    # The match.groupdict() method returns a dictionary with the names and values of the matched groups
    print(match.groupdict())

# Output:
# {'person': 'Satya Nadella', 'role': 'CEO', 'org': 'Microsoft'}


{'person': 'Satya Nadella', 'role': 'CEO', 'org': 'Microsoft'}


- **Implicit Relations**: Relationships inferred indirectly from the context (e.g., "Barack Obama was born in Hawaii" implies a "BornIn" relation).
  - **Code Demonstration**: A more complex method, such as using dependency parsing, is required to infer these relations.


In [6]:
# Import the spaCy library, which is widely used for NLP tasks like tokenization, parsing, and named entity recognition
import spacy

# Load the small English language model.
# "en_core_web_sm" is a pre-trained model provided by spaCy for various NLP tasks, including POS tagging and dependency parsing.
nlp = spacy.load("en_core_web_sm")

# Process the sentence "Barack Obama was born in Hawaii." with the NLP pipeline.
# The `doc` object contains tokens (words), part-of-speech tags, dependencies, etc.
doc = nlp("Barack Obama was born in Hawaii.")

# Iterate through each token (word) in the processed document `doc`.
for token in doc:

    # Check if the current token is the ROOT of the sentence.
    # In a dependency tree, the ROOT is the main verb (or the central action in the sentence).
    if token.dep_ == "ROOT":

        # Print the action (the root verb) and its subject.
        # The subject is generally a noun or pronoun that is related to the ROOT verb.
        # The token itself is the action (verb) and the token's head would refer to its subject (in this case it seems mistaken).
        # However, instead of 'token.head', we should print out the token and also find the actual subject (typically a noun), which is related to the verb.
        print(f"Action: {token.text}, Subject: {token.head.text}")


Action: born, Subject: born


### 6.3 Approaches to Relation Extraction



#### 6.3.1 Pattern-Based Approaches

- **Manual Pattern Design**: Creating hand-crafted rules to identify relations based on specific linguistic patterns, such as using part-of-speech (POS) tags or specific keywords.
  - **Code Demonstration**: Using regular expressions with predefined patterns to identify relations.


In [7]:
import re  # Importing the 're' module to work with regular expressions.

# Defining a regex pattern to match sentences that describe someone being born in a location.
# The pattern uses named groups to capture the person's full name (first and last) and the location.
# - (?P<person>[A-Z][a-z]+ [A-Z][a-z]+): Captures two capitalized words (a first and last name), stored in the 'person' group.
# - (is|was): Matches either 'is' or 'was' (to cover tense variations).
# - (?P<location>[A-Z][a-z]+): Captures a single capitalized word as the location (e.g., "San" from "San Francisco").
# The '\b' ensures that the pattern matches word boundaries so that full words are captured.
pattern = re.compile(r'\b(?P<person>[A-Z][a-z]+ [A-Z][a-z]+)\b (is|was) born in \b(?P<location>[A-Z][a-z]+)\b')

# A test sentence to apply the pattern to.
sentence = "Steve Jobs was born in San Francisco."

# Searches the sentence using the defined regex pattern. It returns a match object if the pattern is found.
match = pattern.search(sentence)

# If a match is found, the named groups ('person' and 'location') are accessed using match.group(),
# and the extracted relation is printed in a formatted way.
if match:
    print(f"Extracted Relation: {match.group('person')} born_in {match.group('location')}")


Extracted Relation: Steve Jobs born_in San


#### 6.3.2 Supervised Learning Methods

- **Feature-Based Classification**: Relationships are identified using traditional classifiers such as Support Vector Machines (SVM) by training models on labeled data. Features may include POS tags, dependency relationships, and named entity types.
  - **Code Demonstration**: Use scikit-learn to create a simple classifier for detecting relationships.


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC  # Importing the Support Vector Classifier (SVC) from the SVM (Support Vector Machine) module.

# A set of sentences (training data), where each sentence describes a relation involving a person (occupation or birthplace).
sentences = ["Satya Nadella is the CEO of Microsoft.", "Tim Cook was born in Alabama."]

# Labels corresponding to the sentences, indicating the type of relation described in each sentence.
labels = ["relation_occupation", "relation_birthplace"]

# Creating an instance of CountVectorizer, which converts text data into a matrix of token counts (bag-of-words model).
vectorizer = CountVectorizer()

# Fitting the vectorizer to the training sentences and transforming them into a sparse matrix of word counts.
X = vectorizer.fit_transform(sentences)

# Initializing the SVC (Support Vector Classifier) with a linear kernel.
# The linear kernel works well with text data when the classes are linearly separable.
classifier = SVC(kernel='linear')

# Training the classifier using the transformed sentences (X) and their corresponding labels.
classifier.fit(X, labels)

# Transforming a new sentence (test data) into the same feature space using the fitted vectorizer.
test_sentence = vectorizer.transform(["Sundar Pichai is the CEO of Google."])

# Using the trained classifier to predict the label for the new test sentence.
prediction = classifier.predict(test_sentence)

# Printing the predicted label for the test sentence.
print(prediction)


['relation_occupation']


- **Deep Learning-Based Models**: Modern methods use neural networks such as Convolutional Neural Networks (CNNs), Long Short-Term Memory (LSTM) networks, or transformers for relation extraction.
  - **Code Demonstration**: Demonstrate a simple LSTM model using PyTorch to classify relations.


In [10]:
import torch  # Importing PyTorch, a deep learning library.
import torch.nn as nn  # Importing the neural network module from PyTorch.
import torch.optim as optim  # Importing optimization algorithms from PyTorch.

# Defining a custom neural network model for relation classification.
class RelationClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RelationClassifier, self).__init__()

        # Embedding layer: Transforms input word indices into dense vectors of a specified size (embedding_dim).
        # - vocab_size: Size of the vocabulary, i.e., how many unique words the model can handle.
        # - embedding_dim: Dimension of the embedding vectors (how many features represent each word).
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layer: A recurrent layer for sequential data, useful for text since word order matters.
        # - embedding_dim: Input size (i.e., the embedding dimension).
        # - hidden_dim: Number of hidden units in the LSTM, which determines the model's capacity.
        # - batch_first=True: Ensures that input shape is (batch_size, sequence_length, embedding_dim).
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Fully connected layer: Maps the LSTM output to the desired number of output classes.
        # - hidden_dim: Input size (coming from LSTM's hidden state).
        # - output_dim: Number of output classes (for classification, this would be the number of relations).
        self.fc = nn.Linear(hidden_dim, output_dim)

    # The forward method defines the forward pass of the neural network.
    def forward(self, x):
        # Embedding: Transforms the input indices into dense vectors of size (batch_size, sequence_length, embedding_dim).
        embedded = self.embedding(x)

        # LSTM: Processes the embedded input through the LSTM layer.
        # - The hidden state (hidden) is passed to the fully connected layer for classification.
        _, (hidden, _) = self.lstm(embedded)

        # Fully connected: The hidden state from the LSTM is used for classification.
        # - hidden.squeeze(0): Removes the unnecessary first dimension of the hidden state (as LSTM returns [1, batch_size, hidden_dim]).
        output = self.fc(hidden.squeeze(0))

        # Returning the output (predictions).
        return output


#### 6.3.3 Distant Supervision

- **Definition**: Using an external knowledge base (e.g., Freebase, Wikidata) to automatically label training data. Relationships in text are aligned with the facts in the database.
  - **Code Demonstration**: Use a set of sample facts to demonstrate distant supervision.


In [11]:
# Sample knowledge base (KB), which holds predefined facts about people and their relationships to organizations.
# Each fact is represented as a dictionary with 'person', 'relation', and 'organization' keys.
knowledge_base = [
    {"person": "Elon Musk", "relation": "CEO_of", "organization": "Tesla"}
]

# Input text from which we want to extract a relationship using distant supervision.
text = "Elon Musk is the CEO of Tesla."

# Function for distant supervision-based relation extraction.
# It tries to find matches between the given text and facts in the knowledge base.
def distant_supervision(text, kb):
    # Iterating over each fact in the knowledge base.
    for fact in kb:
        # Checking if both the person's name and the organization's name from the fact appear in the input text.
        if fact['person'] in text and fact['organization'] in text:
            # If a match is found, print the extracted relation in a human-readable format.
            print(f"Extracted Relation: {fact['person']} is {fact['relation']} {fact['organization']}")

# Calling the distant_supervision function with the input text and knowledge base.
distant_supervision(text, knowledge_base)


Extracted Relation: Elon Musk is CEO_of Tesla


### 6.4 Evaluation of Relation Extraction Models

- **Evaluation Metrics**: Precision, recall, and F1-score are typically used to evaluate the performance of relation extraction models.
  - **Code Demonstration**: Using scikit-learn to evaluate a classifier's performance.


In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score  # Importing precision, recall, and F1-score metrics.

# True labels (ground truth) representing the actual classes for a classification task.
y_true = ["relation_occupation", "relation_birthplace", "relation_occupation"]

# Predicted labels from a classification model.
y_pred = ["relation_occupation", "relation_birthplace", "relation_birthplace"]

# Calculating precision using the micro-averaging method.
# Precision = True Positives / (True Positives + False Positives)
# Micro-averaging computes a global precision by considering all instances across classes,
# treating them as a single binary classification task.
precision = precision_score(y_true, y_pred, average='micro')

# Calculating recall using the micro-averaging method.
# Recall = True Positives / (True Positives + False Negatives)
# Like precision, micro-averaging for recall aggregates the contribution of all classes to compute a global recall.
recall = recall_score(y_true, y_pred, average='micro')

# Calculating F1-score using the micro-averaging method.
# F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
# Micro-averaging for F1 is the harmonic mean of the globally computed precision and recall.
f1 = f1_score(y_true, y_pred, average='micro')

# Printing the calculated precision, recall, and F1-score.
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")


Precision: 0.6666666666666666, Recall: 0.6666666666666666, F1 Score: 0.6666666666666666


### 6.5 Challenges in Relation Extraction

- **Ambiguity in Text**: Ambiguous relations such as a person being associated with multiple organizations can make it challenging to determine the correct relation.
  - **Observation**: Techniques like dependency parsing and contextual embeddings can help disambiguate entity relationships.
  - **Code Demonstration**: Using SpaCy to illustrate dependency parsing and resolve ambiguity.



In [13]:
# Assuming `nlp` is a pre-trained language model (like from spaCy), which processes text.
# 'doc' is the parsed representation of the input text.

doc = nlp("Steve Jobs was the CEO of Apple and Pixar.")

# Iterating over all noun chunks in the document.
# A noun chunk is a noun phrase, which typically includes a noun and its modifiers (e.g., "the CEO of Apple").
for chunk in doc.noun_chunks:
    # Printing the text of the noun chunk and the root (the head noun) of the chunk.
    print(f"Noun Chunk: {chunk.text}, Root: {chunk.root.text}")


Noun Chunk: Steve Jobs, Root: Jobs
Noun Chunk: the CEO, Root: CEO
Noun Chunk: Apple, Root: Apple
Noun Chunk: Pixar, Root: Pixar


- **Lack of Labeled Data**: Annotating text with relationships can be costly and time-consuming.
  - **Observation**: Distant supervision and semi-supervised learning can reduce the dependency on labeled datasets.
  - **Code Demonstration**: Automatically generate labeled examples using distant supervision as shown earlier.



### 6.6 Advanced Techniques for Relation Extraction



#### 6.6.1 Dependency Parsing

- **Definition**: Dependency parsing helps determine the grammatical structure of a sentence, making it easier to identify relations between entities.
  - **Observation**: Parsing dependencies can uncover relationships that are not linear in word order but are instead hierarchical.
  - **Code Demonstration**: Use SpaCy to illustrate dependency parsing and identify relationships.

    ```python
    for token in doc:
        print(f"Token: {token.text}, Dependency: {token.dep_}, Head: {token.head.text}")
    ```



#### 6.6.2 Using Transformers for Relation Extraction

- **Transformers**: BERT and similar transformer-based models are increasingly used for relation extraction due to their ability to capture contextual embeddings.
  - **Observation**: Fine-tuning transformer models like BERT with relation-specific datasets can significantly improve extraction performance.
  - **Code Demonstration**: Example using Hugging Face's transformers library.


In [14]:
from transformers import BertTokenizer, BertForSequenceClassification  # Importing BERT tokenizer and sequence classification model.

# Loading a pre-trained BERT tokenizer. 'bert-base-uncased' is a version of BERT where the text is lowercased.
# The tokenizer converts the input text into a format suitable for the model (tokens and token IDs).
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Loading a pre-trained BERT model for sequence classification.
# 'bert-base-uncased' refers to the BERT model trained on lowercased text, and it's designed for classification tasks.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# The input text that we want to classify.
text = "Jeff Bezos is the founder of Amazon."

# Tokenizing the input text. The tokenizer converts the input text into a set of token IDs that the model understands.
# - return_tensors="pt": This argument returns the tokenized input as PyTorch tensors (since BERT expects tensors as inputs).
inputs = tokenizer(text, return_tensors="pt")

# Passing the tokenized inputs into the BERT model. The model processes the input to produce an output.
# The double asterisks (**inputs) unpack the dictionary of input tensors (e.g., input_ids, attention_mask).
outputs = model(**inputs)

# The output contains logits (raw scores) before applying any activation functions like softmax.
# Logits are the raw, unnormalized predictions of the model, useful for classification tasks.
print(outputs.logits)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[0.3521, 0.1801]], grad_fn=<AddmmBackward0>)


### 6.7 Practical Use Cases of Relation Extraction

- **Knowledge Graph Construction**: Extracting entities and relations to populate a knowledge graph.
  - **Code Demonstration**: Use RDF triples to construct a basic knowledge graph from extracted relations.


In [17]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00


In [18]:
from rdflib import Graph, URIRef, Literal  # Importing necessary classes from the RDFLib library.

# Creating a new RDF graph. An RDF graph is a set of triples, where each triple consists of a subject, predicate, and object.
g = Graph()

# Adding a triple to the graph.
# - The subject is a URI (Unique Resource Identifier) representing "Jeff Bezos".
# - The predicate is a URI representing the relationship "founder_of".
# - The object is a literal, which in this case is the string "Amazon".
g.add((URIRef("http://example.org/Jeff_Bezos"), URIRef("http://example.org/founder_of"), Literal("Amazon")))

# Iterating over all triples (statements) in the graph.
# RDFLib stores triples as a set of (subject, predicate, object) tuples.
for stmt in g:
    # Printing each triple in the graph.
    print(stmt)


(rdflib.term.URIRef('http://example.org/Jeff_Bezos'), rdflib.term.URIRef('http://example.org/founder_of'), rdflib.term.Literal('Amazon'))


- **Question Answering**: Relation extraction is a critical step in understanding relationships between entities to answer complex questions.
  - **Observation**: Question-answering systems use extracted relations to search and return structured answers.
  - **Code Demonstration**: Extracting relations to provide simple QA answers.



In [19]:
# Defining a question about the CEO of Tesla.
question = "Who is the CEO of Tesla?"

# A simple knowledge base (KB) that maps organizations (keys) to their CEO or related information (values).
knowledge_base = {"Tesla": "Elon Musk"}

# Function to answer a question by checking the text and retrieving information from the knowledge base.
def answer_question(q, kb):
    # If the specific phrase "CEO of Tesla" is found in the question, the function attempts to answer.
    if "CEO of Tesla" in q:
        # The function retrieves the value associated with the key "Tesla" from the knowledge base.
        return kb.get("Tesla")

# Calling the function with the provided question and knowledge base.
answer = answer_question(question, knowledge_base)

# Printing the retrieved answer.
print(f"Answer: {answer}")


Answer: Elon Musk
